# Setting up virtual environment

Install Mambaforge https://mamba.readthedocs.io/en/latest/installation.html
If you're on MacOS you can run `brew install mambaforge`

Create a virtual environment for the project
`mamba create -n fakenews python=3.10`

Activate the env `mamba activate fakenews`

Install dependencies we'll need for the project
`mamba install -c huggingface transformers=4.26.0 datasets evaluate jupyterlab scikit-learn`



Run jupyterlab:
`jupyter lab`

In [1]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

/Users/claudio/.conda/envs/fakenews/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([1.], device='mps:0')


/Users/claudio/.conda/envs/fakenews/lib/python3.10/site-packages/torch/_tensor_str.py:115: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/miniforge3/conda-bld/pytorch-recipe_1673797392633/work/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(


In [6]:
from datasets import load_dataset, load_from_disk

In [2]:
dataset = load_dataset("liar")

Dataset liar downloaded and prepared to /Users/claudio/.cache/huggingface/datasets/liar/default/1.0.0/479463e757b7991eed50ffa7504d7788d6218631a484442e2098dabbf3b44514. Subsequent calls will reuse this data.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 795.73it/s]


In [5]:
fox_csv = load_dataset("csv", data_files="164_articles.csv")

NameError: name 'load_dataset' is not defined

In [3]:
fake_csv = load_dataset("csv", data_files="news.csv")

Using custom data configuration default-6f7fc2d25834cfe9


Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 436.50it/s]
Generating train split: 0 examples [00:00, ? examples/s]/Users/claudio/.conda/envs/fakenews/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /Users/claudio/.cache/huggingface/datasets/csv/default-6f7fc2d25834cfe9/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 245.22it/s]


In [9]:
fake_csv

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 6335
    })
})

In [13]:
fake_csv_split = fake_csv["train"].train_test_split(test_size=0.1)

In [14]:
fake_csv_split

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 5701
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 634
    })
})

In [4]:
from transformers import AutoTokenizer

# Load DistilBERT tokenizer and tokenize (encode) the texts
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [5]:
fake_csv_split = fake_csv_split.map(lambda x: {"label": 1 if x["label"] == "FAKE" else 0})

NameError: name 'fake_csv_split' is not defined

In [9]:
# Tokenize and encode the dataset
def tokenize(batch):
    tokenized_batch = tokenizer(batch['body'], padding=True, truncation=True, max_length=128)
    return tokenized_batch

dataset_enc = fox_csv.map(tokenize, remove_columns=['Unnamed: 0', 'title', 'body'], batched=True, num_proc=4)

# Set dataset format for PyTorch
dataset_enc.set_format('torch', columns=['input_ids', 'attention_mask'])

# Check the output
print(dataset_enc["train"].column_names)

Loading cached processed dataset at /Users/claudio/.cache/huggingface/datasets/csv/default-2f9941a8bff5ea48/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-a2f166866ff773b4.arrow
Loading cached processed dataset at /Users/claudio/.cache/huggingface/datasets/csv/default-2f9941a8bff5ea48/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-dd51d0f56476fd8f.arrow
Loading cached processed dataset at /Users/claudio/.cache/huggingface/datasets/csv/default-2f9941a8bff5ea48/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-373fb0646ff6dd03.arrow
Loading cached processed dataset at /Users/claudio/.cache/huggingface/datasets/csv/default-2f9941a8bff5ea48/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-c4c610a9f8c33eb0.arrow


['Url', 'input_ids', 'attention_mask']


In [39]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

# Instantiate a data collator with dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoModelForSequenceClassification

# Load model from checkpoint\n",
model = AutoModelForSequenceClassification.from_pretrained("model",
                                                           num_labels=2)

In [11]:
model.to("mps")

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [17]:
model(**dataset_enc["train"][0])

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [41]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer",  evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [1]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer,device=0)

/Users/claudio/.conda/envs/fakenews/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'model' is not defined

In [2]:
pipe(fox_csv["train"][0]["body"])

NameError: name 'pipe' is not defined

In [42]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [47]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_enc["train"],
    eval_dataset=dataset_enc["test"],
    compute_metrics=compute_metrics,
)

In [48]:
trainer.train()

***** Running training *****
  Num examples = 5701
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2139
  Number of trainable parameters = 66955010


Epoch,Training Loss,Validation Loss,Accuracy
1,0.220600,0.207156,0.938486


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 634
  Batch size = 8


KeyboardInterrupt: 